## Imports and config

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import partridge as ptg
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import datetime

from gtfs_utils import *

alt.renderers.enable('notebook')
alt.data_transformers.enable('json')

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

## Check service improvements

1. frequency improvements
1. service time expansion

קו 8 | מתחנת רכבת האוניברסיטה אל אזור התעסוקה ברמת החייל | תיאום שעות היציאה עם קו 266 ולוחות הזמנים של הרכבת
קו 10 | מראשון לציון לתל אביב | תגבור תדירות
קו 13 | ממסוף כרמלית דרך מרכז תל אביב אל האוניברסיטה | תגבור והארכת שעות פעילות
קו 17 | מגבעת שמואל דרך בני ברק וקניון איילון אל רמת החיל | תגבור והארכת שעות הפעילות
קו 18 | מבקעת אונו לבני ברק | הארכת קו עד מסוף קניון אילון
קו 31 | ממזרח רמת גן, דרך שכונת יד אליהו אל התחנה המרכזית החדשה | תגבור תדירות
קו 23 | מגבעתיים לתל אביב | תגבור תדירות
קו 33 | רמת גן, קניון איילון, שכונות צפון תל אביב ואוניברסיטת תל אביב | הארכת שעות פעילות ותגבור
קו 46 | בין בת ים ומסוף ארלוזורוב | תגבור תדירות
קו 52 | דרום תל אביב, גבעתיים ורמת גן אל קניון איילון ורמת החיל | תגבור תדירות
קו 60 | רמת גן וגבעתיים אל ציר המסגר והתחנה המרכזית בתל אביב | תגבור תדירות
קו 61 | רמת גן וגבעתיים אל תל אביב | תגבור תדירות
קו 63 | משכונת הדר גנים פתח תקווה לתחנת הרכבת סגולה (חב' אפיקים) | יתוגבר ויופעל כל שעות היום
קו 67 | רמת גן, דרך גבעתיים אל מתחם הבורסה וקניון איילון | תגבור תדירות
קו 70 | בסיס תל השומר, בני ברק, גבעת שמואל ורמת גן | תגבור תדירות
קו 92 | יתחיל את מסלולו בתחנה המרכזית פתח תקווה לבני ברק | שינוי מסלול ותגבור
קו 125 | מבת ים לתל אביב | תגבור תדירות
קו 189 | מחולון בכיוון תל אביב ורמת החייל | תגבור תדירות
קו 239 | מרידינג, תחנת רכבת השלום, רחוב יצחק שדה אל שכונת נווה אליעזר | תגבור תדירות
קו 304 | מתחנת רכבת ההגנה אל הכרמלית | תגבור תדירות
הארכת שעות פעילות
קו 5 | תל אביב | הארכת שעות פעילות הקו עד 1:00 בלילה
קו 39 | רמת גן וגבעתיים למרכז תל אביב | הארכת  בשעות פעילות עד 23:00
קו 61 | רמת גן וגבעתיים אל תל אביב | הארכת שעות פעילות הקו עד 1:00 בלילה
קו 166 | מפתח תקווה לתל אביב | הוספת נסיעות לילה עד השעה 1:00 בלילה
קווים חדשים
קו 122 | קו מהיר (חב' דן) ממערב תל אביב ותחנת רכבת האוניברסיטה (שד' רוקח) אל רמת החייל, שיספק קישור איכותי ותדיר לעובדים והמבקרים באזור. הקו יפעל ממסוף כרמלית, דרך הרחובות בוגרשוב, דיזנגוף ושדרות רוקח אל אזור התעסוקה ברמת החייל, משעות הבוקר ועד שעות הערב.
קו 46 | קו חדש המחבר את בת ים, יפו ודרום תל אביב ישירות לרכבת ההגנה ולמרכזי התעסוקה והרפואה ברח' יגאל אלון. הקו מאפשר שירות מהיר יותר לכיוון התחנה המרכזית ורכבת ההגנה דרך הרח' בן צבי ולבון בדרום ת"א וכן לאזור המכללה האקדמאית תל אביב יפו ומתחמי תעסוקה באזור.
קו 238 | מסלול הקו ישונה כך שיעבור מרחוב חשמונאים במסלול מקביל לקו 63 ובכך יאפשר שירות מפתח תקווה למרכז ת"א.
קו 128 | מהעיר תל אביב לאזור התעשייה סגולה וקריית אריה בפתח תקווה
קו 29 | מראש העין, גבעת שמואל לאזור התעשייה קריית אריה וסגולה

In [19]:
dan_lines = ['8', '10', '13', '17', '166', '17', '18', '23', '31', '33', '46', '52', '60', '61', '67', '70', '92', '122', '125', '128', '189', '5', '238', '239', '304']

In [11]:
lines = ['10', '13', '17', '23', '46', '52', '60', '61', '63', '67', '70', '92', '125', '189', '239', '304', '5', '39', '61', '166', '122', '238', '128', '29']

### First Monday of December vs. First Monday of March

In [2]:
!aws s3 cp s3://s3.obus.hasadna.org.il/2017-12-04.zip data/gtfs_feeds/2017-12-04.zip
!aws s3 cp s3://s3.obus.hasadna.org.il/2018-03-05.zip data/gtfs_feeds/2018-03-05.zip

Completed 256.0 KiB/176.1 MiB (143.7 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/176.1 MiB (277.8 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/176.1 MiB (378.3 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/176.1 MiB (496.6 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/176.1 MiB (620.8 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/176.1 MiB (707.5 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/176.1 MiB (786.0 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/176.1 MiB (885.8 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/176.1 MiB (951.7 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/176.1 MiB (1.0 MiB/s) with 1 file(s) remaining    
Completed 2.8 MiB/176.1 MiB (1.1 MiB/s) with 1 file(s) remaining    
Completed 3.0 MiB/176.1 MiB (1.2 MiB/s) with 1 file(s) remaining    
Completed 3.2 MiB/176.1 MiB (1.3 MiB/s) with 1 file(s) remaining    
Completed 3.5 MiB/176.1 MiB (1.3 MiB/s) with 1 file(s) remaining    
Completed 3.8 MiB/176.1 MiB (1.4 M

Completed 256.0 KiB/142.9 MiB (134.4 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/142.9 MiB (264.3 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/142.9 MiB (384.2 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/142.9 MiB (508.2 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/142.9 MiB (598.1 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/142.9 MiB (687.9 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/142.9 MiB (791.2 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/142.9 MiB (898.2 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/142.9 MiB (983.4 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/142.9 MiB (1.1 MiB/s) with 1 file(s) remaining    
Completed 2.8 MiB/142.9 MiB (1.2 MiB/s) with 1 file(s) remaining    
Completed 3.0 MiB/142.9 MiB (1.2 MiB/s) with 1 file(s) remaining    
Completed 3.2 MiB/142.9 MiB (1.3 MiB/s) with 1 file(s) remaining    
Completed 3.5 MiB/142.9 MiB (1.4 MiB/s) with 1 file(s) remaining    
Completed 3.8 MiB/142.9 MiB (1.5 M

### GTFSTK
num_trips -> number_of_trips_per_day, is_loop -> is_circular, start_time -> daily_service_start_time, end_time -> daily_service_end_time, max_headway -> max_minutes_between_buses, min_headway -> min_minutes_between_buses, mean_headway -> average_minutes_between_buses, peak_num_trips -> peak_time_number_of_buses, peak_start_time -> peak_time_start_time, peak_end_time -> peak_time_end_time, service_distance -> service_distance_meters, service_duratoin -> service_duration_minutes, service_speed -> service_speed_kilometer_per_hour, mean_trip_distance -> average_trip_distance_meters, mean_trip_duration -> average_trip_duration_minuets

In [4]:
import gtfstk as gt
import shapely.geometry as sg

In [13]:
dec_path = 'data/gtfs_feeds/2017-12-04.zip'
mar_path = 'data/gtfs_feeds/2018-03-05.zip'

In [16]:
dec_feed = gt.read_gtfs(dec_path, dist_units='m')


In [17]:
dec_trip_stats = dec_feed.compute_trip_stats()

In [18]:
dec_feed = dec_feed.append_dist_to_stop_times(dec_trip_stats)


In [19]:
dates = ['20171204',]

In [20]:
dec_route_stats = dec_feed.compute_route_stats(dec_trip_stats, dates )

In [21]:
mar_feed = gt.read_gtfs(mar_path, dist_units='m')

In [22]:
mar_trip_stats = mar_feed.compute_trip_stats()

In [23]:
mar_feed = mar_feed.append_dist_to_stop_times(mar_trip_stats)


In [31]:
dates = ['20180305',]

In [32]:
mar_route_stats = mar_feed.compute_route_stats(mar_trip_stats, dates )

In [33]:
dec_route_stats.head()

,route_id,route_short_name,route_type,num_trips,num_trip_starts,num_trip_ends,is_loop,is_bidirectional,start_time,end_time,...,mean_headway,peak_num_trips,peak_start_time,peak_end_time,service_distance,service_duration,service_speed,mean_trip_distance,mean_trip_duration,date
0,1,1,3.0,26.0,26.0,26.0,0.0,0.0,05:10:00,21:54:07,...,37.8125,1.0,05:10:00,05:29:07,183170.0,8.283889,22111.595466,7045.0,0.318611,20171204
1,10,4,3.0,35.0,35.0,35.0,0.0,0.0,06:05:00,22:51:36,...,28.0000,2.0,15:55:00,16:06:36,283920.0,12.600000,22533.333333,8112.0,0.360000,20171204
2,10000,91,3.0,1.0,1.0,1.0,0.0,0.0,08:00:00,08:17:31,...,NaN,1.0,08:00:00,08:17:31,6326.0,0.291944,21668.506185,6326.0,0.291944,20171204
3,10001,91,3.0,5.0,5.0,5.0,0.0,0.0,13:00:00,16:51:42,...,52.5000,2.0,13:00:00,13:21:42,41880.0,1.808333,23159.447005,8376.0,0.361667,20171204
4,10002,92,3.0,1.0,1.0,1.0,0.0,0.0,07:30:00,07:58:22,...,NaN,1.0,07:30:00,07:58:22,7156.0,0.472778,15136.075206,7156.0,0.472778,20171204


In [5]:
import pickle
#pickle.dump(dec_route_stats, open('data/171204_route_stats_df.pkl', 'wb'))
#pickle.dump(mar_route_stats, open('data/180305_route_stats_df.pkl', 'wb'))

In [6]:
dec_route_stats = pickle.load(open('data/171204_route_stats_df.pkl', 'rb'))
mar_route_stats = pickle.load(open('data/180305_route_stats_df.pkl', 'rb'))

In [16]:
dec_feed = get_partridge_feed_by_date(dec_path, datetime.date(2017, 12, 4))


In [15]:
mar_feed = get_partridge_feed_by_date(mar_path, datetime.date(2018, 3, 5))


In [7]:
mar_route_stats.head()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5)

In [62]:
m = dec_route_stats[['route_id', 'route_short_name', 'num_trip_starts', 'mean_headway', 'start_time', 'end_time']].merge(mar_route_stats[['route_id', 'route_short_name', 'num_trip_starts', 'mean_headway', 'start_time', 'end_time']], how='left', on=['route_id', 'route_short_name'], suffixes=['_Dec', '_Mar'])

In [63]:
m['num_trip_starts_dif'] = m['num_trip_starts_Mar'] - m['num_trip_starts_Dec']
m['mean_headway_dif'] = m['mean_headway_Mar'] - m['mean_headway_Dec']

In [40]:
m[m.route_short_name.isin(lines)].sort_values(by='num_trip_starts_dif', ascending=False)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5)

In [67]:
named_m = (m[(m.route_short_name.isin(dan_lines))]
 .merge(dec_feed.routes
        .loc[(dec_feed.routes.agency_id=='5'), 
             ['agency_id', 'route_id', 'route_short_name', 'route_long_name']], 
        how='inner'))
named_m['route_short_name'] = pd.to_numeric(named_m.route_short_name)
named_m = named_m.sort_values(by='route_short_name')
named_m = named_m[['route_id', 'route_short_name', 'route_long_name', 'agency_id', 
                   'num_trip_starts_Dec', 'num_trip_starts_Mar', 'num_trip_starts_dif', 
                   'mean_headway_Dec', 'mean_headway_Mar', 'mean_headway_dif',
                  'start_time_Dec', 'start_time_Mar', 'end_time_Dec', 'end_time_Mar']]
named_m

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5)

In [69]:
named_m.to_csv('17-Dec-04VS18-Mar-05.csv')